In [2]:
import pandas as pd
import numpy as np
import requests
import re

## Trabajemos en conformar el Dataframe que englobe a los Homicidios-Hechos

In [76]:
df_homicidiosHechos = pd.read_csv('..//data//homicidios hechos.csv')
df_homicidiosHechos.head(1)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,1/1/2016,2016,1,1,4:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [77]:
# Consultar info del DF
df_homicidiosHechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    int64  
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    int64  
 4   MM                     696 non-null    int64  
 5   DD                     696 non-null    int64  
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Dirección Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    int64  
 15  XY (CA

In [78]:
# Cambiar el nombre de la columna 'N_VICTIMAS' a 'VICTIMAS'
df_homicidiosHechos.rename(columns={'N_VICTIMAS': 'VICTIMAS'}, inplace=True)

In [79]:
# Cambiar el nombre de la columna 'AAAA' a 'AÑO'
df_homicidiosHechos.rename(columns={'AAAA': 'AÑO'}, inplace=True)

In [80]:
# Cambiar el nombre de la columna 'MM' a 'MES'
df_homicidiosHechos.rename(columns={'MM': 'MES'}, inplace=True)

In [81]:
# Cambiar el nombre de la columna 'DD' a 'DIA'
df_homicidiosHechos.rename(columns={'DD': 'DIA'}, inplace=True)

In [82]:
# Cambiar el tipo de datos de la columna 'ID' a str
df_homicidiosHechos['ID'] = df_homicidiosHechos['ID'].astype(str)

In [83]:
# Cambiar el nombre de la columna 'ID' a 'id_siniestro'
df_homicidiosHechos.rename(columns={'ID': 'ID_SINIESTRO'}, inplace=True)

In [84]:
# Identificar y corregir los datos incorrectos en la columna 'HORA' (SD)
df_homicidiosHechos = df_homicidiosHechos[df_homicidiosHechos['HORA'] != 'SD']

In [85]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['HORA'] = df_homicidiosHechos['HORA'].astype(str)

In [86]:
# Normalizar el texto y capitaliza la columna 'LUGAR_DEL_HECHO'
df_homicidiosHechos['LUGAR_DEL_HECHO'] = df_homicidiosHechos['LUGAR_DEL_HECHO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [87]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['LUGAR_DEL_HECHO'] = df_homicidiosHechos['LUGAR_DEL_HECHO'].astype(str)

In [88]:
# Cambiar el nombre de la columna 'LUGAR_DEL_HECHO' a 'DIRECCION'
df_homicidiosHechos.rename(columns={'LUGAR_DEL_HECHO': 'DIRECCION'}, inplace=True)

In [89]:
# Normalizar el texto y capitaliza la columna 'LUGAR_DEL_HECHO'
df_homicidiosHechos['Dirección Normalizada'] = df_homicidiosHechos['Dirección Normalizada'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [90]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['Dirección Normalizada'] = df_homicidiosHechos['Dirección Normalizada'].astype(str)

In [91]:
# Cambiar el nombre de la columna 'Dirección Normalizad' a 'DIRECCION_NORMALIZDA'
df_homicidiosHechos.rename(columns={'Dirección Normalizada': 'DIRECCION_NORMALIZADA'}, inplace=True)

In [92]:
# Cambiar el nombre de la columna 'XY (CABA)' a 'XY_CABA'
df_homicidiosHechos.rename(columns={'XY (CABA)': 'XY_CABA'}, inplace=True)

In [93]:
# Cambiar el tipo de datos de la columna 'ID' a str
df_homicidiosHechos['XY_CABA'] = df_homicidiosHechos['XY_CABA'].astype(str)

In [94]:
# Copiar los valores de la columna 'DIRECCION' a 'DIRECCION_NORMALIZADA' solo si el valor inicial es 'nan'
df_homicidiosHechos.loc[df_homicidiosHechos['DIRECCION_NORMALIZADA'].str.startswith('nan'), 'DIRECCION_NORMALIZADA'] = df_homicidiosHechos['DIRECCION']

In [95]:
# Definir funcion para Normalizar segun USIG la direcciones dadas
def normalizar_direccion_usig(direccion):
    # URL base de la API de Georef
    url_base = "https://apis.datos.gob.ar/georef/api/direcciones"

    # Parámetros de la solicitud GET
    params = {"direccion": direccion}

    # Realizar la solicitud GET a la API de Georef
    response = requests.get(url_base, params=params)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Extraer los datos JSON de la respuesta
        data = response.json()
        
        # Verificar si se encontraron resultados
        if data["cantidad"] > 0:
            # Devolver la dirección normalizada
            return data["direcciones"][0]["ubicacion"]
        else:
            return "No se encontraron resultados"
    else:
        return "Error al realizar la solicitud"  

In [96]:
# Identificar las filas donde 'XY_CABA' es igual a 'Point (. .)'
filas_sd = df_homicidiosHechos['XY_CABA'] == 'Point (. .)'

In [97]:
# Obtener las direcciones normalizadas para las filas donde 'XY_CABA' es igual a 'Point (. .)'
direcciones_normalizadas = df_homicidiosHechos.loc[filas_sd, 'DIRECCION_NORMALIZADA'].apply(normalizar_direccion_usig)

In [98]:
# Reemplazar los valores 'Point (. .)' en 'XY_CABA' con las direcciones normalizadas
df_homicidiosHechos.loc[filas_sd, 'XY_CABA'] = direcciones_normalizadas

In [99]:
# Definir una función para normalizar el texto en la columna XY_CABA
def normalizar_xy_caba(texto):
    if isinstance(texto, dict):
        # Convertir el diccionario a una cadena de texto
        texto = str(texto)
    elif isinstance(texto, float):
        # Convertir el float a una cadena de texto
        texto = str(texto)
    
    # Utilizar una expresión regular para extraer los valores numéricos y eliminar los paréntesis
    valores_numericos = re.findall(r'\d+\.\d+', texto)
    
    # Verificar si se encontraron al menos dos valores numéricos
    if len(valores_numericos) >= 2:
        # Formatear los valores en una cadena normalizada sin paréntesis
        return f"{valores_numericos[0]}, {valores_numericos[1]}"
    else:
        # Si no se encontraron suficientes valores numéricos, devolver una cadena vacía
        return ""   

In [100]:
# Aplicar la función a la columna XY_CABA
df_homicidiosHechos['XY_CABA'] = df_homicidiosHechos['XY_CABA'].apply(normalizar_xy_caba)

In [101]:
# Cambiar el nombre de la columna 'pos X' a 'LONGITUD'
df_homicidiosHechos = df_homicidiosHechos.rename(columns={'pos x': 'LONGITUD'})

In [102]:
# Cambiar el tipo de datos de la columna 'LONGITUD' a str
df_homicidiosHechos['LONGITUD'] = df_homicidiosHechos['LONGITUD'].astype(str)

In [103]:
# Cambiar el nombre de la columna 'pos y' a 'POS_Y'
df_homicidiosHechos.rename(columns={'pos y': 'LATITUD'}, inplace=True)

In [104]:
# Cambiar el tipo de datos de la columna 'LATITUD' a str
df_homicidiosHechos['LATITUD'] = df_homicidiosHechos['LATITUD'].astype(str)

In [105]:
# Normalizar el texto y capitaliza la columna 'PARTICIPANTES'
df_homicidiosHechos['PARTICIPANTES'] = df_homicidiosHechos['PARTICIPANTES'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [106]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['PARTICIPANTES'] = df_homicidiosHechos['PARTICIPANTES'].astype(str)

In [107]:
# Normalizar el texto y capitaliza la columna 'PARTICIPANTES'
df_homicidiosHechos['VICTIMA'] = df_homicidiosHechos['VICTIMA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [108]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['VICTIMA'] = df_homicidiosHechos['VICTIMA'].astype(str)

In [109]:
# Normalizar el texto y capitaliza la columna 'PARTICIPANTES'
df_homicidiosHechos['ACUSADO'] = df_homicidiosHechos['ACUSADO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [110]:
# Cambiar el tipo de datos de la columna 'LUGAR_DEL_HECHO' a str
df_homicidiosHechos['ACUSADO'] = df_homicidiosHechos['ACUSADO'].astype(str)

In [111]:
# Listar de nombres de columnas a eliminar
columnas_a_eliminar = ['FECHA', 'HH', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce']

# Eliminar las columnas especificadas
df_homicidiosHechos = df_homicidiosHechos.drop(columns=columnas_a_eliminar)

In [112]:
df_homicidiosHechos.head(1)

,ID_SINIESTRO,VICTIMAS,AÑO,MES,DIA,HORA,DIRECCION,DIRECCION_NORMALIZADA,COMUNA,XY_CABA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016,1,1,4:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,"Piedra Buena Av. Y Fernandez De La Cruz, F., G...",8,"98896.78238426, 93532.43437792",-58.47533969,-34.68757022,Moto-Auto,Moto,Auto


In [113]:
df_homicidiosHechos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_SINIESTRO           695 non-null    object
 1   VICTIMAS               695 non-null    int64 
 2   AÑO                    695 non-null    int64 
 3   MES                    695 non-null    int64 
 4   DIA                    695 non-null    int64 
 5   HORA                   695 non-null    object
 6   DIRECCION              695 non-null    object
 7   DIRECCION_NORMALIZADA  695 non-null    object
 8   COMUNA                 695 non-null    int64 
 9   XY_CABA                695 non-null    object
 10  LONGITUD               695 non-null    object
 11  LATITUD                695 non-null    object
 12  PARTICIPANTES          695 non-null    object
 13  VICTIMA                695 non-null    object
 14  ACUSADO                695 non-null    object
dtypes: int64(5), object(10)
memo

In [114]:
# Guarda el DataFrame despues del ETL en un archivo CSV
df_homicidiosHechos = df_homicidiosHechos.to_csv('..//data//homicidios_hechos_etl.csv')

## Trabajemos en conformar el Dataframe que englobe a los Homicidios-Victimas

In [220]:
df_homicidiosVictimas = pd.read_csv('..//data//homicidios victimas.csv')
df_homicidiosVictimas.head(1)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1/1/2016,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,1/1/2016


In [223]:
df_homicidiosVictimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_SINIESTRO         717 non-null    object
 1   FECHA                717 non-null    object
 2   AAAA                 717 non-null    int64 
 3   MM                   717 non-null    int64 
 4   DD                   717 non-null    int64 
 5   ROL                  717 non-null    object
 6   VICTIMA              717 non-null    object
 7   SEXO                 717 non-null    object
 8   EDAD                 717 non-null    object
 9   FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(3), object(7)
memory usage: 56.1+ KB


In [221]:
# Cambiar el nombre de la columna 'ID_hecho' a 'id_siniestro'
df_homicidiosVictimas.rename(columns={'ID_hecho': 'ID_SINIESTRO'}, inplace=True)

In [224]:
# Cambiar el nombre de la columna 'AAAA' a 'AÑO'
df_homicidiosVictimas.rename(columns={'AAAA': 'AÑO'}, inplace=True)

In [225]:
# Cambiar el nombre de la columna 'AAAA' a 'AÑO'
df_homicidiosVictimas.rename(columns={'MM': 'MES'}, inplace=True)

In [226]:
# Cambiar el nombre de la columna 'AAAA' a 'AÑO'
df_homicidiosVictimas.rename(columns={'DD': 'DIA'}, inplace=True)

In [227]:
# Normalizar el texto y capitaliza la columna 'ROL'
df_homicidiosVictimas['ROL'] = df_homicidiosVictimas['ROL'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [228]:
# Cambiar el tipo de datos de la columna 'ROL' a str
df_homicidiosVictimas['ROL'] = df_homicidiosVictimas['ROL'].astype(str)

In [229]:
# Normalizar el texto y capitaliza la columna 'ROL'
df_homicidiosVictimas['VICTIMA'] = df_homicidiosVictimas['VICTIMA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [230]:
# Cambiar el tipo de datos de la columna 'ROL' a str
df_homicidiosVictimas['VICTIMA'] = df_homicidiosVictimas['VICTIMA'].astype(str)

In [233]:
# Normalizar el texto y capitaliza la columna 'SEXO'
df_homicidiosVictimas['SEXO'] = df_homicidiosVictimas['SEXO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [234]:
# Cambiar el tipo de datos de la columna 'SEXO' a str
df_homicidiosVictimas['SEXO'] = df_homicidiosVictimas['SEXO'].astype(str)

In [236]:
# Listar de nombres de columnas a eliminar
columnas_a_eliminar = ['FECHA']

# Eliminar las columnas especificadas
df_homicidiosVictimas = df_homicidiosVictimas.drop(columns=columnas_a_eliminar)

In [237]:
df_homicidiosVictimas.head(1)

,ID_SINIESTRO,AÑO,MES,DIA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016,1,1,Conductor,Moto,Masculino,19,1/1/2016


In [238]:
df_homicidiosVictimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_SINIESTRO         717 non-null    object
 1   AÑO                  717 non-null    int64 
 2   MES                  717 non-null    int64 
 3   DIA                  717 non-null    int64 
 4   ROL                  717 non-null    object
 5   VICTIMA              717 non-null    object
 6   SEXO                 717 non-null    object
 7   EDAD                 717 non-null    object
 8   FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(3), object(6)
memory usage: 50.5+ KB


In [239]:
# Guarda el DataFrame despues del ETL en un archivo CSV
df_homicidiosVictimas = df_homicidiosVictimas.to_csv('..//data//homicidios_victimas_etl.csv')

## Trabajemos en conformar el Dataframe que englobe a los Lesiones-Hechos

In [2]:
df_lesionesHechos = pd.read_csv('..//data//lesiones hechos.csv')
df_lesionesHechos.head(1)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,1/1/2019,9:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD


In [3]:
df_lesionesHechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [4]:
# Cambiar el tipo de datos de la columna 'id' a str
df_lesionesHechos['id'] = df_lesionesHechos['id'].astype(str)

In [5]:
# Cambiar el nombre de la columna 'id' a 'ID_LESIONES'
df_lesionesHechos.rename(columns={'id': 'ID_LESIONES'}, inplace=True)

In [6]:
# Cambiar el nombre de la columna 'n_victimas' a 'VICTIMAS'
df_lesionesHechos.rename(columns={'n_victimas': 'VICTIMAS'}, inplace=True)

In [7]:
# Cambiar el nombre de la columna 'aaaa' a 'AÑO'
df_lesionesHechos.rename(columns={'aaaa': 'AÑO'}, inplace=True)

In [8]:
# Cambiar el nombre de la columna 'mm' a 'MES'
df_lesionesHechos.rename(columns={'mm': 'MES'}, inplace=True)

In [9]:
# Cambiar el nombre de la columna 'dd' a 'DIA'
df_lesionesHechos.rename(columns={'dd': 'DIA'}, inplace=True)

In [10]:
# Cambiar el tipo de datos de la columna 'hora' a str
df_lesionesHechos['hora'] = df_lesionesHechos['hora'].astype(str)

In [11]:
# Cambiar el nombre de la columna 'hora' a 'HORA'
df_lesionesHechos.rename(columns={'hora': 'HORA'}, inplace=True)

In [12]:
# Normalizar el texto y capitaliza la columna 'otra_direccion'
df_lesionesHechos['otra_direccion'] = df_lesionesHechos['otra_direccion'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [13]:
# Cambiar el tipo de datos de la columna 'otra_direccion' a str
df_lesionesHechos['otra_direccion'] = df_lesionesHechos['otra_direccion'].astype(str)

In [14]:
# Cambiar el nombre de la columna 'otra_direccion' a 'DIRECCION'
df_lesionesHechos.rename(columns={'otra_direccion': 'DIRECCION'}, inplace=True)

In [15]:
# Normalizar el texto y capitaliza la columna 'direccion_normalizada'
df_lesionesHechos['direccion_normalizada'] = df_lesionesHechos['direccion_normalizada'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [16]:
# Cambiar el tipo de datos de la columna 'otra_direccion' a str
df_lesionesHechos['direccion_normalizada'] = df_lesionesHechos['direccion_normalizada'].astype(str)

In [17]:
# Cambiar el nombre de la columna 'otra_direccion' a 'DIRECCION'
df_lesionesHechos.rename(columns={'direccion_normalizada': 'DIRECCION_NORMALIZADA'}, inplace=True)

In [18]:
# Copiar los valores de la columna 'DIRECCION' a 'DIRECCION_NORMALIZADA' solo si el valor inicial es 'sd'
df_lesionesHechos.loc[df_lesionesHechos['DIRECCION_NORMALIZADA'].str.startswith('Sd'), 'DIRECCION_NORMALIZADA'] = df_lesionesHechos['DIRECCION']

In [19]:
# Copiar los valores de la columna 'DIRECCION_NORMALIZADA' a 'DIRECCION' solo si el valor inicial es 'sd'
df_lesionesHechos.loc[df_lesionesHechos['DIRECCION'].str.startswith('Sd'), 'DIRECCION'] = df_lesionesHechos['DIRECCION_NORMALIZADA']

In [20]:
# Definir funcion para Normalizar segun USIG la direcciones dadas
def normalizar_direccion_usig(direccion):
    # URL base de la API de Georef
    url_base = "https://apis.datos.gob.ar/georef/api/direcciones"

    # Parámetros de la solicitud GET
    params = {"direccion": direccion}

    # Realizar la solicitud GET a la API de Georef
    response = requests.get(url_base, params=params)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Extraer los datos JSON de la respuesta
        data = response.json()
        
        # Verificar si se encontraron resultados
        if data["cantidad"] > 0:
            # Devolver la dirección normalizada
            return data["direcciones"][0]["ubicacion"]
        else:
            return "No se encontraron resultados"
    else:
        return "Error al realizar la solicitud"

In [21]:
# Identificar las filas donde 'geocodificacion_CABA' es igual a 'SD'
filas_sd = df_lesionesHechos['geocodificacion_CABA'] == 'SD'

In [23]:
# Obtener las direcciones normalizadas para las filas donde 'geocodificacion_CABA' es igual a 'SD'
direcciones_normalizadas = df_lesionesHechos.loc[filas_sd, 'DIRECCION_NORMALIZADA'].apply(normalizar_direccion_usig)

In [24]:
# Reemplazar los valores 'SD' en 'geocodificacion_CABA' con las direcciones normalizadas
df_lesionesHechos.loc[filas_sd, 'geocodificacion_CABA'] = direcciones_normalizadas

In [25]:
# Cambiar el nombre de la columna 'geocodificacion_CABA' a 'XY_CABA'
df_lesionesHechos.rename(columns={'geocodificacion_CABA': 'XY_CABA'}, inplace=True)

In [59]:
# Definir una función para normalizar el texto en la columna XY_CABA
def normalizar_xy_caba(texto):
    if isinstance(texto, dict):
        # Convertir el diccionario a una cadena de texto
        texto = str(texto)
    elif isinstance(texto, float):
        # Convertir el float a una cadena de texto
        texto = str(texto)
    
    # Utilizar una expresión regular para extraer los valores numéricos y eliminar los paréntesis
    valores_numericos = re.findall(r'\d+\.\d+', texto)
    
    # Verificar si se encontraron al menos dos valores numéricos
    if len(valores_numericos) >= 2:
        # Formatear los valores en una cadena normalizada sin paréntesis
        return f"{valores_numericos[0]}, {valores_numericos[1]}"
    else:
        # Si no se encontraron suficientes valores numéricos, devolver una cadena vacía
        return ""

In [60]:
# Aplicar la función a la columna XY_CABA
df_lesionesHechos['XY_CABA'] = df_lesionesHechos['XY_CABA'].apply(normalizar_xy_caba)

In [26]:
# Cambiar el nombre de la columna 'longitud' a 'LONGITUD'
df_lesionesHechos.rename(columns={'longitud': 'LONGITUD'}, inplace=True)

In [30]:
# Cambiar el nombre de la columna 'latitud' a 'LATITUD'
df_lesionesHechos.rename(columns={'latutid': 'LATITUD'}, inplace=True)

In [28]:
# Cambiar el nombre de la columna 'victima' a 'VICTIMA'
df_lesionesHechos.rename(columns={'victima': 'VICTIMA'}, inplace=True)

In [64]:
# Normalizar el texto y capitaliza la columna 'direccion_normalizada'
df_lesionesHechos['VICTIMA'] = df_lesionesHechos['VICTIMA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [65]:
# Cambiar el tipo de datos de la columna 'otra_direccion' a str
df_lesionesHechos['VICTIMA'] = df_lesionesHechos['VICTIMA'].astype(str)

In [35]:
# Cambiar el nombre de la columna 'acusado' a 'ACUSADO'
df_lesionesHechos.rename(columns={'acusado': 'ACUSADO'}, inplace=True)

In [67]:
# Normalizar el texto y capitaliza la columna 'direccion_normalizada'
df_lesionesHechos['ACUSADO'] = df_lesionesHechos['ACUSADO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [68]:
# Cambiar el tipo de datos de la columna 'otra_direccion' a str
df_lesionesHechos['ACUSADO'] = df_lesionesHechos['ACUSADO'].astype(str)

In [36]:
# Cambiar el nombre de la columna 'participantes' a 'PARTICIPANTES'
df_lesionesHechos.rename(columns={'participantes': 'PARTICIPANTES'}, inplace=True)

In [69]:
# Normalizar el texto y capitaliza la columna 'direccion_normalizada'
df_lesionesHechos['PARTICIPANTES'] = df_lesionesHechos['PARTICIPANTES'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [70]:
# Cambiar el tipo de datos de la columna 'otra_direccion' a str
df_lesionesHechos['PARTICIPANTES'] = df_lesionesHechos['PARTICIPANTES'].astype(str)

In [34]:
# Listar de nombres de columnas a eliminar
columnas_a_eliminar = ['fecha', 'franja_hora', 'comuna', 'tipo_calle', 'calle', 'altura', 'cruce', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista', 'gravedad']

# Eliminar las columnas especificadas
df_lesionesHechos = df_lesionesHechos.drop(columns=columnas_a_eliminar)

In [71]:
pd.set_option('display.max_columns', None)
df_lesionesHechos.head(1)

,ID_LESIONES,VICTIMAS,AÑO,MES,DIA,HORA,DIRECCION_NORMALIZADA,DIRECCION,XY_CABA,LONGITUD,LATITUD,VICTIMA,ACUSADO,PARTICIPANTES
0,LC-2019-0000179,1,2019,1,1,9:00,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...","Obligado Rafael, Av.Costanera 3300, Barrio: Pa...","104991.46333918849995825, 107720.9664748142677...",-58.408911,-34.559658,Ciclista,Sd,Ciclista-Sd


In [72]:
df_lesionesHechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_LESIONES            23785 non-null  object
 1   VICTIMAS               23785 non-null  int64 
 2   AÑO                    23785 non-null  int64 
 3   MES                    23785 non-null  int64 
 4   DIA                    23785 non-null  int64 
 5   HORA                   23785 non-null  object
 6   DIRECCION_NORMALIZADA  23785 non-null  object
 7   DIRECCION              23785 non-null  object
 8   XY_CABA                23785 non-null  object
 9   LONGITUD               23523 non-null  object
 10  LATITUD                23523 non-null  object
 11  VICTIMA                23785 non-null  object
 12  ACUSADO                23785 non-null  object
 13  PARTICIPANTES          23785 non-null  object
dtypes: int64(4), object(10)
memory usage: 2.5+ MB


In [73]:
# Guarda el DataFrame despues del ETL en un archivo CSV
df_lesionesHechos = df_lesionesHechos.to_csv('..//data//lesiones_hechos_etl.csv')

## Trabajemos en conformar el Dataframe que englobe a los Lesiones-Victimas

In [75]:
df_lesionesVictimas = pd.read_csv('..//data//lesiones victimas.csv')
df_lesionesVictimas

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,1/1/2019,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,1/1/2019,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,1/1/2019,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,1/1/2019,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,1/1/2019,sd,Mujer,45,SD
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,11/9/2021,TRANSPORTE PUBLICO,Varon,87,SD
27601,LC-2021-0530228,2021,10,25,25/10/2021,TRANSPORTE PUBLICO,Mujer,60,SD
27602,LC-2021-0530228,2021,10,25,25/10/2021,TRANSPORTE PUBLICO,Mujer,32,SD
27603,LC-2021-0201378,2021,5,2,2/5/2021,MOTO,Varon,32,SD


In [76]:
df_lesionesVictimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID hecho          27605 non-null  object
 1   AAA               27605 non-null  int64 
 2   MM                27605 non-null  int64 
 3   DD                27605 non-null  int64 
 4   FECHA             27605 non-null  object
 5   VEHICULO_VICTIMA  27605 non-null  object
 6   SEXO              27605 non-null  object
 7   EDAD_VICTIMA      27605 non-null  object
 8   GRAVEDAD          27605 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.9+ MB


In [78]:
# Cambiar el tipo de datos de la columna 'ID hecho' a str
df_lesionesVictimas['ID hecho'] = df_lesionesVictimas['ID hecho'].astype(str)

In [79]:
# Cambiar el nombre de la columna 'ID hecho' a 'ID_LESIONES'
df_lesionesVictimas.rename(columns={'ID hecho': 'ID_LESIONES'}, inplace=True)

In [80]:
# Cambiar el nombre de la columna 'AAAA' a 'AÑO'
df_lesionesVictimas.rename(columns={'AAAAo': 'AÑO'}, inplace=True)

In [81]:
# Cambiar el nombre de la columna 'MM' a 'ID_LESIONES'
df_lesionesVictimas.rename(columns={'MM': 'MES'}, inplace=True)

In [82]:
# Cambiar el nombre de la columna 'DD' a 'DIA'
df_lesionesVictimas.rename(columns={'DD': 'DIA'}, inplace=True)

In [84]:
# Normalizar el texto y capitaliza la columna 'VEHICULO_VICTIMA'
df_lesionesVictimas['VEHICULO_VICTIMA'] = df_lesionesVictimas['VEHICULO_VICTIMA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [85]:
# Cambiar el tipo de datos de la columna 'VEHICULO_VICTIMA' a str
df_lesionesVictimas['VEHICULO_VICTIMA'] = df_lesionesVictimas['VEHICULO_VICTIMA'].astype(str)

In [86]:
# Normalizar el texto y capitaliza la columna 'SEXO'
df_lesionesVictimas['SEXO'] = df_lesionesVictimas['SEXO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

In [87]:
# Cambiar el tipo de datos de la columna 'SEXO' a str
df_lesionesVictimas['SEXO'] = df_lesionesVictimas['SEXO'].astype(str)

In [88]:
# Contar el número de veces que aparece 'SD' en la columna 'GRAVEDAD' 
conteo_sd = df_lesionesVictimas['GRAVEDAD'].value_counts().get('SD', 0)
conteo_sd # Al revisar el valor, se decide eleiminar la columna, ya que no representa porcentualmente peso

20722

In [92]:
# Listar de nombres de columnas a eliminar
columnas_a_eliminar = ['FECHA ', 'GRAVEDAD']

# Eliminar las columnas especificadas
df_lesionesVictimas = df_lesionesVictimas.drop(columns=columnas_a_eliminar)

In [93]:
df_lesionesVictimas.head(1)

,ID_LESIONES,AAA,MES,DIA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA
0,LC-2019-0000053,2019,1,1,Sd,Varon,57


In [94]:
df_lesionesVictimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID_LESIONES       27605 non-null  object
 1   AAA               27605 non-null  int64 
 2   MES               27605 non-null  int64 
 3   DIA               27605 non-null  int64 
 4   VEHICULO_VICTIMA  27605 non-null  object
 5   SEXO              27605 non-null  object
 6   EDAD_VICTIMA      27605 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [95]:
# Guarda el DataFrame despues del ETL en un archivo CSV
df_lesionesVictimas = df_lesionesVictimas.to_csv('..//data//lesiones_victimas_etl.csv')